# Distracted Driver Detection

I am Implemented this project on kaggle and it is gives val_acc of 0.9755.

# Importing Libraries

In [1]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 3 = INFO, WARNING, and ERROR messages are not printed

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

# Import the dataset

In [2]:
dataset = pd.read_csv('../input/driver_imgs_list.csv')
dataset.head(5)

# Building the model

I will develop the model with a total of 3 Convolutional layers, then a Flatten layer and then 3 Dense layers. I'll use the optimizer as **adam**, and loss as **categorical_crossentropy**.

In [3]:
classifier = Sequential()
classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', input_shape = (240, 240, 3), data_format = 'channels_last'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 1024, activation = 'relu'))
classifier.add(Dense(units = 256, activation = 'relu'))
classifier.add(Dense(units = 10, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()

# Creating training data

I will generate more images using **ImageDataGenerator** and split the training data into 80% train and 20% validation split.

In [8]:
train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   validation_split = 0.2)

In [12]:


training_set = train_datagen.flow_from_directory('../input/imgs/train', 
                                                 target_size = (240, 240), 
                                                 batch_size = 32,
                                                 subset = "training")

validation_set = train_datagen.flow_from_directory('../input/imgs/train', 
                                                   target_size = (240, 240), 
                                                   batch_size = 32,
                                                   subset = 'validation')

# Train the model

Using **fit_generator**, I'll train the model.

In [13]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 17943/32,
                         epochs = 10,
                         validation_data = validation_set,
                         validation_steps = 4481/32)